In [ ]:
# Imports and installs

import numpy as np
import pandas as pd
import matplotlib as mpl
import seaborn as sns
from matplotlib import pyplot as plt

#### _SQL for Data Science Capstone Project_

# **2021 Pew Research Center’s American Trends Panel:**

# **Summary of Survey Findings**

---

#### Wave 83: February 2021

#### N = 10,121

# Table of Contents

* [Initial Analysis](#section1)

* [Trump’s Response to COVID-19](#section2)

* [Government Response to COVID-19](#section3)

* [Considerations for Schools Reopening During COVID-19 Pandemic](#section4)

* [COVID-19 Restrictions](#section5)

    * [Number of COVID-19 Restrictions](#section5.1)
    
    
* [Overall Vaccination Status](#section6)   
    
* [Initial Correlation Analysis](#section7)

* [Vaccination Status, Education Level, and Income Level](#section8)
 
* [Vaccination Status and Mask Wearing](#section9)
 
* [Vaccination Status and Religious Views](#section10)
 
* [Vaccination Status and Political Affiliation](#section11)

# Initial Analysis<a class="anchor" id="section1"></a>

In [ ]:
# Read CSV file

df_atp = pd.read_csv('../input/american-trends-panel-wave-83/ATP W83.csv', encoding='Windows-1252', header=0)

In [ ]:
# View first few rows

df_atp.head()

In [ ]:
# View basic statistics

df_atp.describe()

In [ ]:
# Print list of question label and full question

df_atp.loc[0, :].values.tolist()

# Trump’s Response to COVID-19<a class="anchor" id="section2"></a>

In [ ]:
def create_and_clean_dataframe(*args):
    
    '''
    Returns new dataframe. Creates new dataframe with args, drops first row, replaces NaN with 0,
    changes data type to string and then float, calculates weight and adds new column, groups data
    and adds new column for total count, adds new column for weighted total, and returns new dataframe.
    
    Should be used with survey question that has ONE part.

            Parameters:
                    args: columns (col) needed from original dataframe (df_atp)

            Returns:
                    pandas dataframe
    '''
    
    # Create list of columns to use in dataframe

    list_of_columns = []
    
    for col in args:       
  
        list_of_columns.append(col)
    
        new_df = df_atp[list_of_columns]
        
    # Drop first row with question

    new_df = new_df.drop(new_df.index[0])

    # Replace NaN with 0

    new_df = new_df.fillna(0)
    
    # Change data type to string and then float

    new_df = new_df.astype(str)
    new_df = new_df.astype(float)
    
    # Calculate weight    

    calc_weight = new_df.groupby(list_of_columns[-1]) \
                        .apply(lambda x: (x[list_of_columns[0]] * \
                                          x[list_of_columns[-1]]).sum() / x[list_of_columns[-1]].sum())
    
    # Join new column on dataframe

    new_df = new_df.join(calc_weight.rename(('Weight')), on=list_of_columns[0])

    # Group and add column with count

    new_df_count = new_df.groupby([list_of_columns[0], 'Weight'])[list_of_columns[0]].count().reset_index(name='Total')

    # MAKE SURE TYPE IS FLOAT AND ROUNDED TO AVOID CALCULATION ERRORS
    
    new_df_count['Weight'] = new_df_count['Weight'].astype(float)
    new_df_count['Weight'] = new_df_count['Weight'].round(2)
    new_df_count['Total'] = new_df_count['Total'].astype(float)
    new_df_count['Total'] = new_df_count['Total'].round(2)
    
    # Add columnn for weighted total

    new_df_count['Weighted Total'] = new_df_count['Weight'] * new_df_count['Total']
    
    # Return new dataframe
        
    return new_df_count

In [ ]:
# Create new dataframe

df_trump_count = create_and_clean_dataframe('COVIDEGFPDT_W83', 'WEIGHT_W83')

df_trump_count

In [ ]:
def add_description(row, col_name):
    
    '''
    Returns rating based on value in col_name. Use in lambda function with pandas .apply to add column to dataframe.
    
            Parameters:
                    row: row in pandas dataframe
                    col_name: column name with numeric rating

            Returns:
                    Rating based on value in col_name (excellent, good, only fair, poor, or no answer)
    '''
    
    if row[col_name] == 1:
        return 'Excellent'
    if row[col_name] == 2:
        return 'Good'
    if row[col_name] == 3:
        return 'Only fair'
    if row[col_name] == 4:
        return 'Poor'
    if row[col_name] == 99:
        return 'No answer'

In [ ]:
# Create new dataframe

df_trump_count['Rating'] = df_trump_count.apply(lambda row: add_description(row, 'COVIDEGFPDT_W83'), axis=1)

df_trump_count

In [ ]:
def create_bar_chart(df, saved_file):    
    
    '''
    Returns bar chart with two subplots produced from dataframe. Defines fig and axes. Defines subplots.
    Sets fonts and spacing. Sets text for x-axis tick labels. Sets text for titles and axis labels.
    Adds labels to bars to show numbers. Sets rotation of x-axis tick labels. Shows box and grid.
    Exports and displays bar chart.
    
            Parameters:
                    df: dataframe to use for chart
                    saved_file: file name to use for exported image file

            Returns:
                    Bar chart with two subplots
    '''
    
    # Define fig and axes
    
    fig, axes = plt.subplots(1,2, figsize=(12, 6))

    # Define subplots
    
    ax1 = df['Total'].plot(ax=axes[0], kind='bar', grid=True, color='darkred', zorder=2)
    ax2 = df['Weighted Total'].plot(ax=axes[1], kind='bar', grid=True, color='darkred', zorder=2)
       
    # Set fonts
    
    mpl.rcParams['font.sans-serif'] = 'Century Gothic'
    mpl.rcParams['font.family'] = 'sans-serif'
            
    # Set the spacing between subplots
    
    plt.subplots_adjust(wspace=0.3,
                        hspace=0.3)
    
    # Set text for subplot titles
    
    ax1.title.set_text('Unweighted')
    ax2.title.set_text('Weighted')
    
    # Set text for x-axis tick labels (show Rating category rather than number)
    
    ax1.set_xticklabels(df['Rating'])
    ax2.set_xticklabels(df['Rating'])
    
    # Set text for x-axis and y-axis labels
    
    ax1.set_xlabel('Rating')
    ax1.set_ylabel('Total')
    
    ax2.set_xlabel('Rating')
    ax2.set_ylabel('Total')
    
    # Add labels to bars to show numbers
    
    ax1.bar_label(ax1.containers[0], label_type='edge')
    ax2.bar_label(ax2.containers[0], label_type='edge')
    
    # Set rotation of x-axis tick labels
    
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=360)
    ax2.set_xticklabels(ax2.get_xticklabels(), rotation=360)
    
    # Show box and grid
    
    plt.box(True)
    plt.grid(True)
        
    # Save chart (export); set bbox_inches='tight' so image is not cut off
    
    plt.savefig(saved_file, dpi=300, bbox_inches='tight')
        
    # Display chart
    
    plt.show()

In [ ]:
# View bar chart

create_bar_chart(df_trump_count,
                 'trump.png')

# Government Response to COVID-19<a class="anchor" id="section3"></a>

In [ ]:
def create_and_clean_dataframe_mult_cols(*args):
    
    '''
    Returns new dataframe. Creates new dataframe with args, drops first row, replaces NaN with 0,
    changes data type to string and then float, calculates weight and adds new columns, groups data
    and adds new columns for total count, adds new columns for weighted total, and returns new dataframe.
    
    Should be used with survey question that has MULTIPLE parts (e.g., a, b, c, d).

            Parameters:
                    args: columns (col) needed from original dataframe (df_atp)

            Returns:
                    pandas dataframe
    '''

    # Create list of columns to use in dataframe
    
    list_of_columns = []
    
    for col in args:
  
        list_of_columns.append(col)
    
        new_df = df_atp[list_of_columns]
        
    # Drop first row with question

    new_df = new_df.drop(new_df.index[0])    

    # Replace NaN with 0

    new_df = new_df.fillna(0)
    
    # Change data type to string and then float    

    new_df = new_df.astype(str)
    new_df = new_df.astype(float)

    # Calculate weight for each part
    
    for column in new_df.iloc[0:, 0:-1]:    

        calc_weight = new_df.groupby(list_of_columns[-1]) \
                            .apply(lambda x: (x[column] * \
                                              x[list_of_columns[-1]]).sum() / x[list_of_columns[-1]].sum())
        
        # Join new columns on dataframe

        new_df = new_df.join(calc_weight.rename(('Weight for ' + column)), on=list_of_columns[0])
        
        # MAKE SURE TYPE IS FLOAT AND ROUNDED TO AVOID CALCULATION ERRORS

        new_df['Weight for ' + column] = new_df['Weight for ' + column].astype(float)
        new_df['Weight for ' + column] = new_df['Weight for ' + column].round(2)
        
    # Drop weight column

    new_df = new_df.drop('WEIGHT_W83', axis=1)
    
    # Calculate count and add columns
    
    col_range = len(list_of_columns) - 1
    for column in new_df.iloc[0:, 0:col_range]:
        
        total = new_df.groupby(column) \
                               .apply(lambda x: x[column].count())
        
        # Join new columns on dataframe

        new_df = new_df.join(total.rename(('Total for ' + column)), on=list_of_columns[0])   

        # MAKE SURE TYPE IS FLOAT AND ROUNDED TO AVOID CALCULATION ERRORS

        new_df['Total for ' + column] = new_df['Total for ' + column].astype(float)
        new_df['Total for ' + column] = new_df['Total for ' + column].round(2)
        
        # Add columns for weighted totals for each part

        new_df['Weighted Total for '+ column] = new_df['Weight for ' + column] * new_df['Total for ' + column]
        
        # MAKE SURE TYPE IS FLOAT AND ROUNDED TO AVOID CALCULATION ERRORS

        new_df['Weighted Total for '+ column] = new_df['Weighted Total for '+ column].astype(float)
        new_df['Weighted Total for '+ column] = new_df['Weighted Total for '+ column].round(2)        
        
    # Return new dataframe
   
    return new_df

In [ ]:
# Create new dataframe

df_gov_count = create_and_clean_dataframe_mult_cols('COVIDEGFP_a_W83',
                                                    'COVIDEGFP_b_W83',
                                                    'COVIDEGFP_c_W83',
                                                    'COVIDEGFP_d_W83',
                                                    'COVIDEGFP_g_W83',
                                                    'WEIGHT_W83')

df_gov_count.head(10)

In [ ]:
def create_pivot_table_mult_answers(df, index_column, *args):    
    
    '''
    Returns new dataframe with totals (unweighted and weighted values) grouped by rating for each question part.
    
            Parameters:
                    df: original dataframe to use
                    index_column: column needed for index
                    args: columns (col) needed for values

            Returns:
                    pandas dataframe
    '''
    
    # Create list of columns to use in dataframe (values)
    
    myList = []
    
    for col in args:
        
        myList.append(col)
        
    # Create pivot table
  
    df_pivot = pd.pivot_table(df, index=index_column,
                                  values=myList,
                                  aggfunc='first',
                                  fill_value=0) \
                 .astype(int)
            
    # Rename index header
    
    df_pivot.index.names = ['Rating']

    # Stack pivot table

    df_pivot = df_pivot.stack()
    
    # Unstack Rating column
    
    df_pivot = df_pivot.unstack(0)

    # Return dataframe
    
    return df_pivot

In [ ]:
# Create new dataframe

df_gov_count_pivot = create_pivot_table_mult_answers(df_gov_count, 'COVIDEGFP_a_W83',
                                                    'Total for COVIDEGFP_a_W83',
                                                    'Weighted Total for COVIDEGFP_a_W83',
                                                    'Total for COVIDEGFP_b_W83',
                                                    'Weighted Total for COVIDEGFP_b_W83',
                                                    'Total for COVIDEGFP_c_W83',
                                                    'Weighted Total for COVIDEGFP_c_W83',
                                                    'Total for COVIDEGFP_d_W83',
                                                    'Weighted Total for COVIDEGFP_d_W83',
                                                    'Total for COVIDEGFP_g_W83',
                                                    'Weighted Total for COVIDEGFP_g_W83')

df_gov_count_pivot

In [ ]:
# Rename columns and index

df_gov_count_pivot = df_gov_count_pivot.rename(columns={1.0: 'Excellent'})
df_gov_count_pivot = df_gov_count_pivot.rename(columns={2.0: 'Good'})
df_gov_count_pivot = df_gov_count_pivot.rename(columns={3.0: 'Only fair'})
df_gov_count_pivot = df_gov_count_pivot.rename(columns={4.0: 'Poor'})
df_gov_count_pivot = df_gov_count_pivot.rename(columns={99.0: 'Refused to answer'})

df_gov_count_pivot = df_gov_count_pivot.rename(index={'Total for COVIDEGFP_a_W83':
                                                      'Total for Part a'})
df_gov_count_pivot = df_gov_count_pivot.rename(index={'Total for COVIDEGFP_b_W83':
                                                      'Total for Part b'})
df_gov_count_pivot = df_gov_count_pivot.rename(index={'Total for COVIDEGFP_c_W83':
                                                      'Total for Part c'})
df_gov_count_pivot = df_gov_count_pivot.rename(index={'Total for COVIDEGFP_d_W83':
                                                      'Total for Part d'})
df_gov_count_pivot = df_gov_count_pivot.rename(index={'Total for COVIDEGFP_g_W83':
                                                      'Total for Part g'})

df_gov_count_pivot = df_gov_count_pivot.rename(index={'Weighted Total for COVIDEGFP_a_W83':
                                                      'Weighted Total for Part a'})
df_gov_count_pivot = df_gov_count_pivot.rename(index={'Weighted Total for COVIDEGFP_b_W83':
                                                      'Weighted Total for Part b'})
df_gov_count_pivot = df_gov_count_pivot.rename(index={'Weighted Total for COVIDEGFP_c_W83':
                                                      'Weighted Total for Part c'})
df_gov_count_pivot = df_gov_count_pivot.rename(index={'Weighted Total for COVIDEGFP_d_W83':
                                                      'Weighted Total for Part d'})
df_gov_count_pivot = df_gov_count_pivot.rename(index={'Weighted Total for COVIDEGFP_g_W83':
                                                      'Weighted Total for Part g'})

In [ ]:
# View heat map

df_gov_count_pivot.style.background_gradient(cmap='Reds')

In [ ]:
def create_bar_chart_mult_bars(df,
                               ax1_start,
                               ax1_stop,
                               ax2_start,
                               ax2_stop,
                               y_axis_limit,
                               saved_file):
    
    '''
    Returns bar chart with two subplots produced from dataframe. Defines fig and axes. Defines subplots.
    Sets fonts and spacing. Sets text for titles and axis labels. Sets custom y-axis limit values. 
    Sets legend to represent both subplots. Shows box and grid. Exports and displays bar chart.
    
            Parameters:
                    df: dataframe to use for chart
                    ax1_start: starting value for first subplot (sliced data)
                    ax1_stop: stopping value for first subplot (sliced data)
                    ax2_start: starting value for second subplot (sliced data)
                    ax2_stop: stopping value for second subplot (sliced data)
                    y_axis_limit: custom limit for y-axis
                    saved_file: file name to use for exported image file

            Returns:
                    Bar chart with two subplots and grouped bars
    '''
    
    # Define fig and axes    
    
    fig, axes = plt.subplots(1,2, figsize=(12,6))
    
    # Define subplots

    ax1 = df.iloc[ax1_start:ax1_stop, 0:].plot(ax=axes[0], kind='bar', grid=True, cmap='RdYlBu', zorder=2, legend=False)
    ax2 = df.iloc[ax2_start:ax2_stop, 0:].plot(ax=axes[1], kind='bar', grid=True, cmap='RdYlBu', zorder=2, legend=False)
    
    # Set fonts
    
    mpl.rcParams['font.sans-serif'] = 'Century Gothic'
    mpl.rcParams['font.family'] = 'sans-serif'
            
    # Set the spacing between subplots
    
    plt.subplots_adjust(wspace=0.3,
                        hspace=0.6)
    
    # Set text for subplot titles

    ax1.title.set_text('Unweighted')
    ax2.title.set_text('Weighted')    
   
    # Set text for x-axis and y-axis labels
    
    ax1.set_xlabel('Question')
    ax1.set_ylabel('Total')
    
    ax2.set_xlabel('Question')
    ax2.set_ylabel('Total')
        
    # Define custom y-axis limit values (limit should be the same for both subplots)
    
    custom_ylim = (0, y_axis_limit)

    # Set the values for all axes
    
    plt.setp(ax1, ylim=custom_ylim)
    plt.setp(ax2, ylim=custom_ylim)
    
    # Define handles and labels and set legend to represent both subplots
    
    handles, labels = ax2.get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper right', bbox_to_anchor=(1.1, 0.8))
  
    # Show box and grid
    
    plt.box(True)
    plt.grid(True)
    
    # Save chart (export); set bbox_inches='tight' so image is not cut off   
    
    plt.savefig(saved_file, dpi=300, bbox_inches='tight')
    
    # Display chart
    
    plt.show()

In [ ]:
# View bar chart

create_bar_chart_mult_bars(df_gov_count_pivot,
                           0,
                           5,
                           5,
                           10,
                           20000,
                           'gov.png')

# Considerations for Schools Reopening During COVID-19 Pandemic<a class="anchor" id="section4"></a>

In [ ]:
# Create new dataframe

df_school_count = create_and_clean_dataframe_mult_cols('SCHLREOPEN_a_W83',
                                                       'SCHLREOPEN_b_W83',
                                                       'SCHLREOPEN_c_W83',
                                                       'SCHLREOPEN_d_W83',
                                                       'SCHLREOPEN_e_W83',
                                                       'SCHLREOPEN_f_W83',
                                                       'WEIGHT_W83')

df_school_count.head(20)

In [ ]:
# Create new dataframe

df_school_count_pivot_all = create_pivot_table_mult_answers(df_school_count, 'SCHLREOPEN_a_W83',
                                                           'Total for SCHLREOPEN_a_W83',
                                                           'Weighted Total for SCHLREOPEN_a_W83',
                                                           'Total for SCHLREOPEN_b_W83',
                                                           'Weighted Total for SCHLREOPEN_b_W83',
                                                           'Total for SCHLREOPEN_c_W83',
                                                           'Weighted Total for SCHLREOPEN_c_W83',
                                                           'Total for SCHLREOPEN_d_W83',
                                                           'Weighted Total for SCHLREOPEN_d_W83',
                                                           'Total for SCHLREOPEN_e_W83',
                                                           'Weighted Total for SCHLREOPEN_e_W83',
                                                           'Total for SCHLREOPEN_f_W83',
                                                           'Weighted Total for SCHLREOPEN_f_W83')

df_school_count_pivot_all

In [ ]:
# Drop column with no responses

df_school_count_pivot_all = df_school_count_pivot_all.drop([99.0], axis=1)

df_school_count_pivot_all

In [ ]:
# Rename columns and index

df_school_count_pivot_all = df_school_count_pivot_all.rename(columns={1.0: 'A lot'})
df_school_count_pivot_all = df_school_count_pivot_all.rename(columns={2.0: 'Some'})
df_school_count_pivot_all = df_school_count_pivot_all.rename(columns={3.0: 'Not too much'})
df_school_count_pivot_all = df_school_count_pivot_all.rename(columns={4.0: 'None at all'})

df_school_count_pivot_all = df_school_count_pivot_all.rename(index={'Total for SCHLREOPEN_a_W83':
                                                                    'Total for Part a',
                                                                    'Total for SCHLREOPEN_b_W83':
                                                                    'Total for Part b',
                                                                    'Total for SCHLREOPEN_c_W83':
                                                                    'Total for Part c',
                                                                    'Total for SCHLREOPEN_d_W83':
                                                                    'Total for Part d',
                                                                    'Total for SCHLREOPEN_e_W83':
                                                                    'Total for Part e',
                                                                    'Total for SCHLREOPEN_f_W83':
                                                                    'Total for Part f',
                                                                    'Weighted Total for SCHLREOPEN_a_W83':
                                                                    'Weighted Total for Part a',
                                                                    'Weighted Total for SCHLREOPEN_b_W83':
                                                                    'Weighted Total for Part b',
                                                                    'Weighted Total for SCHLREOPEN_c_W83':
                                                                    'Weighted Total for Part c',
                                                                    'Weighted Total for SCHLREOPEN_d_W83':
                                                                    'Weighted Total for Part d',
                                                                    'Weighted Total for SCHLREOPEN_e_W83':
                                                                    'Weighted Total for Part e',
                                                                    'Weighted Total for SCHLREOPEN_f_W83':
                                                                    'Weighted Total for Part f'})

In [ ]:
# View heat map

df_school_count_pivot_all.style.background_gradient(cmap='Reds')

In [ ]:
# View bar chart

create_bar_chart_mult_bars(df_school_count_pivot_all,
                           0,
                           6,
                           6,
                           12,
                           12000,
                           'school_all.png')

In [ ]:
# Create new dataframe

df_school_reopen = create_and_clean_dataframe('TCHRVACCINE_W83', 'WEIGHT_W83')

df_school_reopen

In [ ]:
def add_description_reopen(row, col_name):  
    
    '''
    Returns rating based on value in col_name. Use in lambda function with pandas .apply to add column to dataframe.
    
            Parameters:
                    row: row in pandas dataframe
                    col_name: column name with numeric rating

            Returns:
                    Rating based on value in col_name (reopen as soon as possible, wait to reopen,
                    or no answer)
    '''
    
    if row[col_name] == 1:
        return 'Reopen as soon as possible'
    if row[col_name] == 2:
        return 'Wait to reopen'
    if row[col_name] == 99:
        return 'No answer'

In [ ]:
# Create new dataframe

df_school_reopen['Rating'] = df_school_reopen.apply(lambda row: add_description_reopen(row, 'TCHRVACCINE_W83'), axis=1)

df_school_reopen

In [ ]:
# View bar chart

create_bar_chart(df_school_reopen, 'school_reopen.png')

# COVID-19 Restrictions<a class="anchor" id="section5"></a>

In [ ]:
# Create new dataframe

df_restrict_count = create_and_clean_dataframe_mult_cols('COVID_RESTRICTION_a_W83',
                                                         'COVID_RESTRICTION_c_W83',
                                                         'COVID_RESTRICTION_e_W83',
                                                         'COVID_RESTRICTION_f_W83',
                                                         'COVID_RESTRICTION_h_W83',
                                                         'WEIGHT_W83')

df_restrict_count

In [ ]:
# Create new dataframe

df_restrict_count_pivot = create_pivot_table_mult_answers(df_restrict_count, 'COVID_RESTRICTION_a_W83',
                                                          'Total for COVID_RESTRICTION_a_W83',
                                                           'Weighted Total for COVID_RESTRICTION_a_W83',
                                                           'Total for COVID_RESTRICTION_c_W83',
                                                           'Weighted Total for COVID_RESTRICTION_c_W83',
                                                           'Total for COVID_RESTRICTION_e_W83',
                                                           'Weighted Total for COVID_RESTRICTION_e_W83',
                                                           'Total for COVID_RESTRICTION_f_W83',
                                                           'Weighted Total for COVID_RESTRICTION_f_W83',
                                                           'Total for COVID_RESTRICTION_h_W83',
                                                           'Weighted Total for COVID_RESTRICTION_h_W83')

df_restrict_count_pivot

In [ ]:
# Rename columns and index

df_restrict_count_pivot = df_restrict_count_pivot.rename(columns={1.00: 'Necessary'})
df_restrict_count_pivot = df_restrict_count_pivot.rename(columns={2.00: 'Unnecessary'})

df_restrict_count_pivot = df_restrict_count_pivot.rename(index={'Total for COVID_RESTRICTION_a_W83':
                                                                'Total for Part a'})
df_restrict_count_pivot = df_restrict_count_pivot.rename(index={'Total for COVID_RESTRICTION_c_W83':
                                                                'Total for Part c'})
df_restrict_count_pivot = df_restrict_count_pivot.rename(index={'Total for COVID_RESTRICTION_e_W83':
                                                                'Total for Part e'})
df_restrict_count_pivot = df_restrict_count_pivot.rename(index={'Total for COVID_RESTRICTION_f_W83':
                                                                'Total for Part f'})
df_restrict_count_pivot = df_restrict_count_pivot.rename(index={'Total for COVID_RESTRICTION_h_W83':
                                                                'Total for Part h'})

df_restrict_count_pivot = df_restrict_count_pivot.rename(index={'Weighted Total for COVID_RESTRICTION_a_W83':
                                                                'Weighted Total for Part a'})
df_restrict_count_pivot = df_restrict_count_pivot.rename(index={'Weighted Total for COVID_RESTRICTION_c_W83':
                                                                'Weighted Total for Part c'})
df_restrict_count_pivot = df_restrict_count_pivot.rename(index={'Weighted Total for COVID_RESTRICTION_e_W83':
                                                                'Weighted Total for Part e'})
df_restrict_count_pivot = df_restrict_count_pivot.rename(index={'Weighted Total for COVID_RESTRICTION_f_W83':
                                                                'Weighted Total for Part f'})
df_restrict_count_pivot = df_restrict_count_pivot.rename(index={'Weighted Total for COVID_RESTRICTION_h_W83':
                                                                'Weighted Total for Part h'})

df_restrict_count_pivot

In [ ]:
# View heat map

df_restrict_count_pivot.style.background_gradient(cmap='Reds')

In [ ]:
# Drop ratings of 98 or 99 (translation error and no answer)

df_restrict_count_pivot_dropped = df_restrict_count_pivot.drop([98.0, 99.0], axis=1)

df_restrict_count_pivot_dropped

In [ ]:
# View bar chart

create_bar_chart_mult_bars(df_restrict_count_pivot_dropped,
                           0,
                           5,
                           5,
                           10,
                           12000,
                           'restrict.png')

## Number of COVID-19 Restrictions<a class="anchor" id="section5.1"></a>

In [ ]:
# Create new dataframe

df_num_restrict_count = create_and_clean_dataframe('COVID_OPENMORE_W83', 'WEIGHT_W83')

df_num_restrict_count

In [ ]:
def add_description_num_restrict(row, col_name):
    
    '''
    Returns rating based on value in col_name. Use in lambda function with pandas .apply to add column to dataframe.
    
            Parameters:
                    row: row in pandas dataframe
                    col_name: column name with numeric rating

            Returns:
                    Rating based on value in col_name (more, fewer, about the same number, or no answer)
    '''

    if row[col_name] == 1:
        return 'More'
    if row[col_name] == 2:
        return 'Fewer'
    if row[col_name] == 3:
        return 'About the same number'
    if row[col_name] == 99:
        return 'No answer'

In [ ]:
# Create new dataframe

df_num_restrict_count.apply(lambda row: add_description_num_restrict(row, 'COVID_OPENMORE_W83'), axis=1)

df_num_restrict_count['Rating'] = df_num_restrict_count.apply(lambda row: add_description_num_restrict(row,
                                                                                                       'COVID_OPENMORE_W83'), 
                                                              axis=1)

df_num_restrict_count

In [ ]:
# View bar chart

create_bar_chart(df_num_restrict_count,
                 'restrict_no.png')

# Overall Vaccination Status<a class="anchor" id="section6"></a>

In [ ]:
# Create new dataframe

df_vacc = create_and_clean_dataframe('COVID_VAXD_W83', 'WEIGHT_W83')

df_vacc

In [ ]:
def add_description_vacc(row, col_name):
    
    '''
    Returns rating based on value in col_name. Use in lambda function with pandas .apply to add column to dataframe.
    
            Parameters:
                    row: row in pandas dataframe
                    col_name: column name with numeric rating

            Returns:
                    Rating based on value in col_name (yes or no)
    ''' 
    
    if row[col_name] == 1.00:
        return 'Yes'
    if row[col_name] == 2.00:
        return 'No'

In [ ]:
# Create new dataframe

df_vacc['Rating'] = df_vacc.apply(lambda row: add_description_vacc(row, 'COVID_VAXD_W83'), axis=1)

df_vacc

In [ ]:
# View bar chart

df_vacc_chart = create_bar_chart(df_vacc,
                                 'vacc.png')

# Initial Correlation Analysis<a class="anchor" id="section7"></a>

In [ ]:
# Create new dataframe

df_demographic_corr = create_and_clean_dataframe_mult_cols('COVID_VAXD_W83',
                                                           'COVIDMASK1_W83',
                                                           'F_AGECAT',
                                                           'F_GENDER',
                                                           'F_EDUCCAT',
                                                           'F_EDUCCAT2',
                                                           'F_INC_SDT1',
                                                           'F_INC_TIER2',
                                                           'F_RELIG',
                                                           'F_PARTY_FINAL',
                                                           'WEIGHT_W83')

In [ ]:
# Create new dataframe and rename columns

df_demographic_corr_renamed = df_demographic_corr.rename(columns={'COVID_VAXD_W83':'Vaccination Status',
                                                                  'COVIDMASK1_W83': 'Mask Wearing',
                                                                  'F_AGECAT':'Age',
                                                                  'F_GENDER':'Gender',
                                                                  'F_EDUCCAT':'Education 1',
                                                                  'F_EDUCCAT2':'Education 2',
                                                                  'F_INC_SDT1':'Family Income',
                                                                  'F_INC_TIER2':'Income Tier',
                                                                  'F_RELIG':'Religion',
                                                                  'F_PARTY_FINAL':'Political Party',
                                                                  'WEIGHT_W83':'Weight for Vaccination Status'})

df_demographic_corr_renamed

In [ ]:
def create_dataframe_corr(heatmap_title, saved_file, *args):
    
    '''
    Returns new dataframe for Seaborn heatmap with unweighted data to show correlations.

            Parameters:
                    heatmap_title: heatmap title
                    args: columns (col) needed from original dataframe (df_demographic_corr_renamed)
                    saved_file: file name to use for exported image file

            Returns:
                    New dataframe for Seaborn heatmap
    '''
    
    # Create list of columns to use in heatmap

    list_of_columns = []
    
    for col in args: 
        
        list_of_columns.append(col)
        
        new_df = df_demographic_corr_renamed[list_of_columns]

    # Drop first row with question

    new_df = new_df.drop(new_df.index[0])

    # Replace NaN with 0

    new_df = new_df.fillna(0)

    # Convert numbers to floats

    new_df = new_df.astype(float)

    # Set options for displaying floats (only up to 2 digits)

    pd.options.display.float_format = '{:.2f}'.format
    
    # Compute pairwise correlation of columns
      
    new_df.corr()
    
    # Define plot for figure    
    
    plt.figure(figsize = (12, 8))
    
    # Set title for figure
    
    plt.title(heatmap_title)
    
    # Create heatmap
    
    sns.heatmap(new_df.corr(), annot=True, cmap='Reds')
    
    # Save heatmap (export); set bbox_inches='tight' so image is not cut off
    
    plt.savefig(saved_file, dpi=300, bbox_inches='tight')
    
    # Show heatmap
    
    plt.show()
    
    # Return new dataframe
      
    return new_df

In [ ]:
# Create new dataframe

df_user_corr = create_dataframe_corr('Initial Correlation Analysis',
                                     'corr_all.png',
                                     'Vaccination Status',
                                     'Mask Wearing',
                                     'Age',
                                     'Gender',
                                     'Education 1',
                                     'Education 2',
                                     'Family Income',
                                     'Income Tier',
                                     'Religion',
                                     'Political Party')

# Vaccination Status, Education Level, and Income Level<a class="anchor" id="section8"></a>

In [ ]:
# Create new dataframe

df_vacc_edu_inc_corr = create_dataframe_corr('Correlation Analysis: Vaccination Status, Education, and Income',
                                             'corr_edu_inc.png',
                                             'Vaccination Status',
                                             'Education 1',
                                             'Education 2',
                                             'Family Income',
                                             'Income Tier')

In [ ]:
# Create new dataframe

df_vacc_edu1_corr = df_demographic_corr_renamed[['Education 1', 'Vaccination Status', 'Weight for COVID_VAXD_W83']]
                                                 
df_vacc_edu1_corr                                                                            

In [ ]:
# Create function

def create_corr_pivot_table(df, seg_column):    
    
    '''
    Returns new pivot table to analyze correlations with vaccination status.

            Parameters:            
                    df: dataframe to use for pivot table
                    seg_column: column to use in index of pivot table (specific demographic to use in analysis)

            Returns:
                    New pivot table
    '''
    
    # Create pivot table

    pivot_df = pd.pivot_table(df, index=[seg_column, 'Vaccination Status'],
                              aggfunc={'Weight for COVID_VAXD_W83': 'first',
                                       'Vaccination Status': 'count'},
                              fill_value=0) \
                 .astype(float) \
                 .rename(columns={'Vaccination Status': 'Total (Unweighted)',
                                  'Weight for COVID_VAXD_W83': 'Weight'})
    
    # Add column and calculate total weighted values

    pivot_df['Total (Weighted)'] = pivot_df['Total (Unweighted)'] * pivot_df['Weight']
    
    # Drop 'Weight' column

    pivot_df = pivot_df.drop('Weight', axis=1)
    
    # Return new pivot table

    return pivot_df

In [ ]:
# Create pivot table

df_vacc_edu1_corr_pivot = create_corr_pivot_table(df_vacc_edu1_corr, 'Education 1')

In [ ]:
# Rename index

df_vacc_edu1_corr_pivot.index.names = ['Education Level', 'Vaccination Status']

# Replace responses with responses

df_vacc_edu1_corr_pivot = df_vacc_edu1_corr_pivot.rename(index={1.0: 'College graduate+'}, level=0)
df_vacc_edu1_corr_pivot = df_vacc_edu1_corr_pivot.rename(index={2.0: 'Some college'}, level=0)
df_vacc_edu1_corr_pivot = df_vacc_edu1_corr_pivot.rename(index={3.0: 'High school graduate or less'}, level=0)
df_vacc_edu1_corr_pivot = df_vacc_edu1_corr_pivot.rename(index={99.0: 'Refused to answer'}, level=0)

df_vacc_edu1_corr_pivot = df_vacc_edu1_corr_pivot.rename(index={1.00: 'Yes'}, level=1)
df_vacc_edu1_corr_pivot = df_vacc_edu1_corr_pivot.rename(index={2.00: 'No'}, level=1)
df_vacc_edu1_corr_pivot = df_vacc_edu1_corr_pivot.rename(index={99.00: 'Refused to answer'}, level=1)

In [ ]:
# Change data type to integer

df_vacc_edu1_corr_pivot = df_vacc_edu1_corr_pivot.astype(int)

# View heat map

df_vacc_edu1_corr_pivot.style.background_gradient(cmap='Reds')

In [ ]:
# Create new dataframe

df_vacc_edu2_corr = df_demographic_corr_renamed[['Vaccination Status', 'Education 2', 'Weight for COVID_VAXD_W83']]

In [ ]:
# Create pivot table

df_vacc_edu2_corr_pivot = create_corr_pivot_table(df_vacc_edu2_corr, 'Education 2')

In [ ]:
# Rename index

df_vacc_edu2_corr_pivot.index.names = ['Education Level', 'Vaccination Status']

# Replace responses with responses

df_vacc_edu2_corr_pivot = df_vacc_edu2_corr_pivot.rename(index={1.00: 'Less than high school '}, level=0)
df_vacc_edu2_corr_pivot = df_vacc_edu2_corr_pivot.rename(index={2.00: 'High school graduate'}, level=0)
df_vacc_edu2_corr_pivot = df_vacc_edu2_corr_pivot.rename(index={3.00: 'Some college, no degree'}, level=0)
df_vacc_edu2_corr_pivot = df_vacc_edu2_corr_pivot.rename(index={4.00: 'Associate’s degree'}, level=0)
df_vacc_edu2_corr_pivot = df_vacc_edu2_corr_pivot.rename(index={5.00: 'College graduate/some postgrad'}, level=0)
df_vacc_edu2_corr_pivot = df_vacc_edu2_corr_pivot.rename(index={6.00: 'Postgraduate'}, level=0)
df_vacc_edu2_corr_pivot = df_vacc_edu2_corr_pivot.rename(index={99.00: 'Refused to answer'}, level=0)

df_vacc_edu2_corr_pivot = df_vacc_edu2_corr_pivot.rename(index={1.00: 'Yes'}, level=1)
df_vacc_edu2_corr_pivot = df_vacc_edu2_corr_pivot.rename(index={2.00: 'No'}, level=1)
df_vacc_edu2_corr_pivot = df_vacc_edu2_corr_pivot.rename(index={99.00: 'Refused to answer'}, level=1)

In [ ]:
# Create new dataframe

df_vacc_inc_corr = df_demographic_corr_renamed[['Vaccination Status', 'Family Income', 'Weight for COVID_VAXD_W83']]

In [ ]:
# Create pivot table

df_vacc_inc_corr_pivot = create_corr_pivot_table(df_vacc_inc_corr, 'Family Income')

In [ ]:
# Change data type to integer

df_vacc_edu2_corr_pivot = df_vacc_edu2_corr_pivot.astype(int)

# View heat map

df_vacc_edu2_corr_pivot.style.background_gradient(cmap='Reds')

In [ ]:
# Rename index

df_vacc_inc_corr_pivot.index.names = ['Family Income (in US$)', 'Vaccination Status']

# Replace responses with responses

df_vacc_inc_corr_pivot = df_vacc_inc_corr_pivot.rename(index={1.0: 'Less than 30,000'}, level=0)
df_vacc_inc_corr_pivot = df_vacc_inc_corr_pivot.rename(index={2.0: '30,000 to less than 40,000'}, level=0)
df_vacc_inc_corr_pivot = df_vacc_inc_corr_pivot.rename(index={3.0: '40,000 to less than 50,000'}, level=0)
df_vacc_inc_corr_pivot = df_vacc_inc_corr_pivot.rename(index={4.0: '50,000 to less than 60,000'}, level=0)
df_vacc_inc_corr_pivot = df_vacc_inc_corr_pivot.rename(index={5.0: '60,000 to less than 70,000'}, level=0)
df_vacc_inc_corr_pivot = df_vacc_inc_corr_pivot.rename(index={6.0: '70,000 to less than 80,000'}, level=0)
df_vacc_inc_corr_pivot = df_vacc_inc_corr_pivot.rename(index={7.0: '80,000 to less than 90,000'}, level=0)
df_vacc_inc_corr_pivot = df_vacc_inc_corr_pivot.rename(index={8.0: '90,000 to less than 100,000'}, level=0)
df_vacc_inc_corr_pivot = df_vacc_inc_corr_pivot.rename(index={8.0: '90,000 to less than 100,000'}, level=0)
df_vacc_inc_corr_pivot = df_vacc_inc_corr_pivot.rename(index={9.0: '100,000 or more'}, level=0)
df_vacc_inc_corr_pivot = df_vacc_inc_corr_pivot.rename(index={99.0: 'Refused to answer'}, level=0)

df_vacc_inc_corr_pivot = df_vacc_inc_corr_pivot.rename(index={1.00: 'Yes'}, level=1)
df_vacc_inc_corr_pivot = df_vacc_inc_corr_pivot.rename(index={2.00: 'No'}, level=1)
df_vacc_inc_corr_pivot = df_vacc_inc_corr_pivot.rename(index={99.00: 'Refused to answer'}, level=1)

In [ ]:
# Change data type to integer

df_vacc_inc_corr_pivot = df_vacc_inc_corr_pivot.astype(int)

# View heat map

df_vacc_inc_corr_pivot.style.background_gradient(cmap='Reds')

In [ ]:
# Create new dataframe

df_vacc_inctier_corr = df_demographic_corr_renamed[['Vaccination Status', 'Income Tier', 'Weight for COVID_VAXD_W83']]

In [ ]:
# Create pivot table

df_vacc_inctier_corr_pivot = create_corr_pivot_table(df_vacc_inctier_corr, 'Income Tier')

In [ ]:
# Rename index

df_vacc_inctier_corr_pivot.index.names = ['Income Tier', 'Vaccination Status']

# Replace responses with responses

df_vacc_inctier_corr_pivot = df_vacc_inctier_corr_pivot.rename(index={1.0: 'Lower income'}, level=0)
df_vacc_inctier_corr_pivot = df_vacc_inctier_corr_pivot.rename(index={2.0: 'Middle income'}, level=0)
df_vacc_inctier_corr_pivot = df_vacc_inctier_corr_pivot.rename(index={3.0: 'Upper income'}, level=0)
df_vacc_inctier_corr_pivot = df_vacc_inctier_corr_pivot.rename(index={99.0: 'Refused to answer'}, level=0)

df_vacc_inctier_corr_pivot = df_vacc_inctier_corr_pivot.rename(index={1.00: 'Yes'}, level=1)
df_vacc_inctier_corr_pivot = df_vacc_inctier_corr_pivot.rename(index={2.00: 'No'}, level=1)
df_vacc_inctier_corr_pivot = df_vacc_inctier_corr_pivot.rename(index={99.00: 'Refused to answer'}, level=1)

In [ ]:
# Change data type to integer

df_vacc_inctier_corr_pivot = df_vacc_inctier_corr_pivot.astype(int)

# View heat map

df_vacc_inctier_corr_pivot.style.background_gradient(cmap='Reds')

# Vaccination Status and Mask Wearing<a class="anchor" id="section9"></a>

In [ ]:
# Create new dataframe

df_mask_corr = df_demographic_corr_renamed[['Vaccination Status', 'Mask Wearing', 'Weight for COVID_VAXD_W83']]

In [ ]:
# Create pivot table

df_mask_corr_pivot = create_corr_pivot_table(df_mask_corr, 'Mask Wearing')

In [ ]:
# Rename index

df_mask_corr_pivot.index.names = ['Mask Wearing', 'Vaccination Status']

# Replace responses with responses

df_mask_corr_pivot = df_mask_corr_pivot.rename(index={1.00: 'All or most of the time'}, level=0)
df_mask_corr_pivot = df_mask_corr_pivot.rename(index={2.00: 'Some of the time'}, level=0)
df_mask_corr_pivot = df_mask_corr_pivot.rename(index={3.00: 'Hardly ever'}, level=0)
df_mask_corr_pivot = df_mask_corr_pivot.rename(index={4.00: 'Never'}, level=0)
df_mask_corr_pivot = df_mask_corr_pivot.rename(index={5.00: 'Have not gone to these types of places'}, level=0)
df_mask_corr_pivot = df_mask_corr_pivot.rename(index={99.00: 'Refused to answer'}, level=0)

df_mask_corr_pivot = df_mask_corr_pivot.rename(index={1.00: 'Yes'}, level=1)
df_mask_corr_pivot = df_mask_corr_pivot.rename(index={2.00: 'No'}, level=1)
df_mask_corr_pivot = df_mask_corr_pivot.rename(index={99.00: 'Refused to answer'}, level=1)

In [ ]:
# Change data type to integer

df_mask_corr_pivot = df_mask_corr_pivot.astype(int)

# View heat map

df_mask_corr_pivot.style.background_gradient(cmap='Reds')

# Vaccination Status and Religious Views<a class="anchor" id="section10"></a>

In [ ]:
# Create new dataframe

df_rel_corr = df_demographic_corr_renamed[['Vaccination Status', 'Religion', 'Weight for COVID_VAXD_W83']]

In [ ]:
# Create pivot table

df_rel_corr_pivot = create_corr_pivot_table(df_rel_corr, 'Religion')

In [ ]:
# Rename index

df_rel_corr_pivot.index.names = ['Religion', 'Vaccination Status']

# Replace responses with responses

df_rel_corr_pivot = df_rel_corr_pivot.rename(index={1.00: 'Protestant'}, level=0)
df_rel_corr_pivot = df_rel_corr_pivot.rename(index={2.00: 'Roman Catholic'}, level=0)
df_rel_corr_pivot = df_rel_corr_pivot.rename(index={3.00: 'Mormon'}, level=0)
df_rel_corr_pivot = df_rel_corr_pivot.rename(index={4.00: 'Orthodox'}, level=0)
df_rel_corr_pivot = df_rel_corr_pivot.rename(index={5.00: 'Jewish'}, level=0)
df_rel_corr_pivot = df_rel_corr_pivot.rename(index={6.00: 'Muslim'}, level=0)
df_rel_corr_pivot = df_rel_corr_pivot.rename(index={7.00: 'Buddhist'}, level=0)                            
df_rel_corr_pivot = df_rel_corr_pivot.rename(index={8.00: 'Hindu'}, level=0)
df_rel_corr_pivot = df_rel_corr_pivot.rename(index={9.00: 'Atheist'}, level=0)                            
df_rel_corr_pivot = df_rel_corr_pivot.rename(index={10.00: 'Agnostic'}, level=0)                            
df_rel_corr_pivot = df_rel_corr_pivot.rename(index={11.00: 'Other'}, level=0)
df_rel_corr_pivot = df_rel_corr_pivot.rename(index={12.00: 'Nothing in particular'}, level=0)      
df_rel_corr_pivot = df_rel_corr_pivot.rename(index={99.00: 'Refused to answer'}, level=0)

df_rel_corr_pivot = df_rel_corr_pivot.rename(index={1.00: 'Yes'}, level=1)
df_rel_corr_pivot = df_rel_corr_pivot.rename(index={2.00: 'No'}, level=1)
df_rel_corr_pivot = df_rel_corr_pivot.rename(index={99.00: 'Refused to answer'}, level=1)

In [ ]:
# Change data type to integer

df_rel_corr_pivot = df_rel_corr_pivot.astype(int)

# View heat map

df_rel_corr_pivot.style.background_gradient(cmap='Reds')

# Vaccination Status and Political Affiliation<a class="anchor" id="section11"></a>

In [ ]:
# Create new dataframe

df_pol_corr = df_demographic_corr_renamed[['Vaccination Status', 'Political Party', 'Weight for COVID_VAXD_W83']]

In [ ]:
# Create pivot table

df_pol_corr_pivot = create_corr_pivot_table(df_pol_corr, 'Political Party')

In [ ]:
# Rename index

df_pol_corr_pivot.index.names = ['Political Party', 'Vaccination Status']

# Replace responses with responses

df_pol_corr_pivot = df_pol_corr_pivot.rename(index={1.00: 'Republican'}, level=0)
df_pol_corr_pivot = df_pol_corr_pivot.rename(index={2.00: 'Democrat'}, level=0)
df_pol_corr_pivot = df_pol_corr_pivot.rename(index={3.00: 'Independent'}, level=0)
df_pol_corr_pivot = df_pol_corr_pivot.rename(index={4.00: 'Something else'}, level=0) 
df_pol_corr_pivot = df_pol_corr_pivot.rename(index={99.00: 'Refused to answer'}, level=0)

df_pol_corr_pivot = df_pol_corr_pivot.rename(index={1.00: 'Yes'}, level=1)
df_pol_corr_pivot = df_pol_corr_pivot.rename(index={2.00: 'No'}, level=1)
df_pol_corr_pivot = df_pol_corr_pivot.rename(index={99.00: 'Refused to answer'}, level=1)

In [ ]:
# Change data type to integer

df_pol_corr_pivot = df_pol_corr_pivot.astype(int)

# View heat map

df_pol_corr_pivot.style.background_gradient(cmap='Reds')